In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from pprint import pprint
from pydantic import Field, conint, conlist

from pydantic_nested.model import create_nested_model


schema = {
    "str1": str,
    "str2": (str, "default_string"),
    "str3": (str | None, None),
    "str4": (str, Field("str4 default value", description="q4 description")),
    "list1": list,
    "list2": list | tuple,
    "list3": list[int | str],
    "list4": (list, None),
    "list5": (list[float], Field(default_factory=list)),
    "dict_list1": {
        "str1": (str | None, None),
        "str2": (str, "default_string"),
    },
    "dict_list2": {"member_ids": conlist(int, min_length=1, max_length=20)},
    "dict_list3": {"include": {"member_ids": list[conint(ge=1)]}},
    "dict_list_nullable": (
        {
            "str1": (str | None, None),
            "str2": (str, "default_string"),
        },
        None,
    ),  # Need to make key nullable
    "list_of_objects1": [{"str1": str}],
    "list_of_objects2": [{"str1": str}, {"bool1": bool}],
    "list_of_objects3": ([{"str1": str}, {"bool1": bool}], None),
    "list_of_objects4": ([{"str1": str}, {"bool1": bool}], Field(default_factory=list, max_length=5))
}

# inner = get_args(schema["q3"])[0]
# print((inner))
# for wtf in get_args(inner):
#     print(wtf)
# exit()
# Create the model dynamically
DynamicModel = create_nested_model(schema, "Test")
# exit()

# Example of using the dynamically created model with the full query encapsulated
sample = {
    "str1": "t1",
    "list1": [1, 2],
    "list2": [1, 2, 3],
    "list3": ["1", 2],
    "dict_list1": {},
    "dict_list2": {"member_ids": [1, 2, 3]},
    "dict_list3": {"include": {"member_ids": [1, 2, 3]}},
    "ifnorable": (
        1,
        23,
    ),
    "list_of_objects1": [{"str1": "nice"}],
    "list_of_objects2": [{"bool1": True}]
}

create_sample = True
if create_sample:
    model_instance = DynamicModel(**sample)
    # print(DynamicModel)
    print(type(model_instance))
    print("-" * 50)
    pprint(model_instance.model_dump(mode="json"))
    print("-" * 50)


# model_json = DynamicModel.model_json_schema()
# referenced = jsonref.replace_refs(model_json, proxies=False)
# print(referenced.keys())
# referenced.pop("$defs")
# # print(referenced)
# # referenced.encode()
# print(json.dumps(referenced))

<class 'pydantic_nested.model.TestModel'>
--------------------------------------------------
{'dict_list1': {'str1': None, 'str2': 'default_string'},
 'dict_list2': {'member_ids': [1, 2, 3]},
 'dict_list3': {'include': {'member_ids': [1, 2, 3]}},
 'dict_list_nullable': None,
 'list1': [1, 2],
 'list2': [1, 2, 3],
 'list3': ['1', 2],
 'list4': None,
 'list5': [],
 'list_of_objects1': [{'str1': 'nice'}],
 'list_of_objects2': [{'bool1': True}],
 'list_of_objects3': None,
 'list_of_objects4': [],
 'str1': 't1',
 'str2': 'default_string',
 'str3': None,
 'str4': 'str4 default value'}
--------------------------------------------------


In [3]:
from pydantic import BaseModel, create_model, Field
from typing import get_args, get_origin


class wtf(BaseModel):
    str1: str | None = None
    str2: str = "defailt_string"


class Test(BaseModel):
    a: wtf = None


fields = {
    "str1": (str | None, None),
    "str2": (str, "default_string"),
    "st3": (Test, None),
}
ok = create_model("lolas", **{"a": (Test, Field(default_factory=list))})
print(get_origin(fields))

None


In [ ]:
query_schema = {
    "str2": (str, "default_string"),
    "str3": (str | None, None),
    "str4": (str, Field("str4 default value", description="str4 description")),
    "list1": list,
    "list2": list | tuple,
    "list3": list[int | str],
    "list4": (list, None),
    "list5": (list[float], Field(default_factory=list)),
    "dict1": {
        "str1": (str | None, None),
        "str2": (str, "default_string"),
    },
    "dict2": {"product_ids": conlist(int, min_length=1, max_length=20)},
    "dict3": {"include": {"product_ids": list[conint(ge=1)]}},
    "dict_list_nullable": (
        {
            "str1": (str | None, None),
            "str2": (str, "default_string"),
        },
        None,
    ),
    "list_of_objects1": [{"str1": str}],
    "list_of_objects2": [{"str1": str}, {"bool1": bool}],
    "list_of_objects3__?": ([{"str1": str}, {"bool1": bool}], None),
    "list_of_objects4": ([{"str1": str}, {"bool1": bool}], Field(default_factory=list, max_length=5)),
}

TestModel = create_nested_model(query_schema, "TestModel")

###
sample_data = {
    "str1": "example string",
    "list1": [1, 2, 3],
    "list2": (1, 2),
    "list3": ["1", 2],
    "dict1": {},
    "dict2": {"product_ids": [1, 2, 3]},
    "dict3": {"include": {"product_ids": [1, 2, 3]}},
    "list_of_objects1": [{"str1": "object1"}],
    "list_of_objects2": [{"bool1": True}],
}

# Instantiate the model and validate the data
model_instance = TestModel(**sample_data)
pprint(model_instance.model_dump(mode="json"))

{'dict1': {'str1': None, 'str2': 'default_string'},
 'dict2': {'product_ids': [1, 2, 3]},
 'dict3': {'include': {'product_ids': [1, 2, 3]}},
 'dict_list_nullable': None,
 'list1': [1, 2, 3],
 'list2': [1, 2],
 'list3': ['1', 2],
 'list4': None,
 'list5': [],
 'list_of_objects1': [{'str1': 'object1'}],
 'list_of_objects2': [{'bool1': True}],
 'list_of_objects3': None,
 'list_of_objects4': [],
 'str2': 'default_string',
 'str3': None,
 'str4': 'str4 default value'}
